In [1]:
from datetime import datetime
import pandas as pd
sales_data = pd.read_excel('Data files for UniMelb Project - data until 2021/Sales Data.xlsx')
stateless_data = pd.read_excel('Data files for UniMelb Project - data until 2021/stateless.xlsx')
sales_data

,Sold-To Party,Name 1,Material,Description,Delivery Date,Confirmed Quantity,Sales Unit,Plant,Ship-To Party,Ship-to party name 1
0,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2021-02-01,8.0,DRM,S106,64679,WIRRINA WTP
1,11611,Trility Pty Ltd,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-12-21,4.0,DRM,S106,57395,Birdwood WWTP
2,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-12-18,8.0,DRM,S106,64679,WIRRINA WTP
3,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-11-12,8.0,DRM,S106,64679,WIRRINA WTP
4,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-11-12,6.0,DRM,S106,64679,WIRRINA WTP
...,...,...,...,...,...,...,...,...,...,...
228427,7650,Bartter Enterprises,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-12-17,1615.0,L,N106,37418,CHICKEN PLANT-- Steggles /Bartter
228428,6190,YMCA Singleton Swim & Fitness,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-12-17,1650.0,L,N106,6190,YMCA Singleton Swim & Fitness
228429,8029,Paper Australia Pty Ltd,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-11-28,1100.0,L,V127,40241,AP Maryvale Mill - Cooling Towers
228430,7650,Bartter Enterprises,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-01-08,839.0,L,N106,37417,TURKEY PLANT- Steggles /Bartter


In [2]:
# Covert data type
sales_data['Sold-To Party']= sales_data['Sold-To Party'].astype(str)
sales_data['Sold-To Party']= sales_data['Sold-To Party'].astype(str)

In [3]:
# Collecting stateless corps
stateless_corp = []
for i in stateless_data['SH_Customer No']:
    stateless_corp.append(i)

In [4]:
# Drop data with 0 confirmed quantity and corps which are not in Australia
sales_data = sales_data.drop(sales_data[sales_data['Confirmed Quantity']==0].index) 
sales_data = sales_data.drop(sales_data[sales_data['Sold-To Party'].isin(stateless_corp)].index)

In [5]:
sales_data['Quantity in L'] = 'undefined' # Initialise 

In [6]:
sales_data.head()

,Sold-To Party,Name 1,Material,Description,Delivery Date,Confirmed Quantity,Sales Unit,Plant,Ship-To Party,Ship-to party name 1,Quantity in L
0,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2021-02-01,8.0,DRM,S106,64679,WIRRINA WTP,undefined
1,11611,Trility Pty Ltd,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-12-21,4.0,DRM,S106,57395,Birdwood WWTP,undefined
2,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-12-18,8.0,DRM,S106,64679,WIRRINA WTP,undefined
3,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-11-12,8.0,DRM,S106,64679,WIRRINA WTP,undefined
4,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-11-12,6.0,DRM,S106,64679,WIRRINA WTP,undefined


In [7]:
#Convert KG into TO
sales_data.loc[sales_data['Sales Unit']=='KG', 'Confirmed Quantity'] = sales_data['Confirmed Quantity']/1000
sales_data.loc[sales_data['Sales Unit']=='KG', 'Sales Unit'] = 'TO'

In [8]:
#Convert all units into L
sales_data.loc[sales_data['Sales Unit']=='DRM', 'Quantity in L'] = sales_data['Confirmed Quantity']*200
sales_data.loc[sales_data['Sales Unit']=='BOX', 'Quantity in L'] = sales_data['Confirmed Quantity']*1000
sales_data.loc[sales_data['Sales Unit']=='PLY', 'Quantity in L'] = sales_data['Confirmed Quantity']*15
sales_data.loc[sales_data['Sales Unit']=='IBC', 'Quantity in L'] = sales_data['Confirmed Quantity']*1000
sales_data.loc[sales_data['Sales Unit']=='KL', 'Quantity in L'] = sales_data['Confirmed Quantity']*1000
sales_data.loc[sales_data['Sales Unit']=='L', 'Quantity in L'] = sales_data['Confirmed Quantity']*1
sales_data.loc[(sales_data['Sales Unit']=='PLS') & (sales_data['Material'].str.contains('MHL-22.5PAIL')), 'Quantity in L'] = sales_data['Confirmed Quantity']*15
sales_data.loc[(sales_data['Sales Unit']=='PLS') & (sales_data['Material'].str.contains('PAC23-15')), 'Quantity in L'] = sales_data['Confirmed Quantity']*15
sales_data.loc[(sales_data['Sales Unit']=='PLS') & (sales_data['Material'].str.contains('SODHYP12PM-12.5')), 'Quantity in L'] = sales_data['Confirmed Quantity']*12.5
sales_data.loc[(sales_data['Sales Unit']=='PLS') & (sales_data['Material'].str.contains('SODHYP12PM-15')), 'Quantity in L'] = sales_data['Confirmed Quantity']*15
sales_data.loc[(sales_data['Sales Unit']=='TO') & (sales_data['Material'].str.contains('ALUSUL08')), 'Quantity in L'] = sales_data['Confirmed Quantity']*(1000/1.31)
sales_data.loc[(sales_data['Sales Unit']=='TO') & (sales_data['Material'].str.contains('MHL')), 'Quantity in L'] = sales_data['Confirmed Quantity']*(1000/1.5)
sales_data.loc[(sales_data['Sales Unit']=='TO') & (sales_data['Material'].str.contains('PAC10')), 'Quantity in L'] = sales_data['Confirmed Quantity']*(1000/1.19)
sales_data.loc[(sales_data['Sales Unit']=='TO') & (sales_data['Material'].str.contains('PAC23')), 'Quantity in L'] = sales_data['Confirmed Quantity']*(1000/1.34)
sales_data.loc[(sales_data['Sales Unit']=='TO') & (sales_data['Material'].str.contains('SODHYP')), 'Quantity in L'] = sales_data['Confirmed Quantity']*(1000/1.2)

In [9]:
sales_data['Delivery Date'] = sales_data['Delivery Date'].dt.strftime('%Y-%m')

In [10]:
sales_data

,Sold-To Party,Name 1,Material,Description,Delivery Date,Confirmed Quantity,Sales Unit,Plant,Ship-To Party,Ship-to party name 1,Quantity in L
0,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2021-02,8.0,DRM,S106,64679,WIRRINA WTP,1600
1,11611,Trility Pty Ltd,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-12,4.0,DRM,S106,57395,Birdwood WWTP,800
2,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-12,8.0,DRM,S106,64679,WIRRINA WTP,1600
3,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-11,8.0,DRM,S106,64679,WIRRINA WTP,1600
4,9477,SA Water Corporation,ALUSUL08-200,"ALUMINIUM SULFATE LIQUID, 200L drum",2020-11,6.0,DRM,S106,64679,WIRRINA WTP,1200
...,...,...,...,...,...,...,...,...,...,...,...
228427,7650,Bartter Enterprises,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-12,1615.0,L,N106,37418,CHICKEN PLANT-- Steggles /Bartter,1615
228428,6190,YMCA Singleton Swim & Fitness,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-12,1650.0,L,N106,6190,YMCA Singleton Swim & Fitness,1650
228429,8029,Paper Australia Pty Ltd,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-11,1100.0,L,V127,40241,AP Maryvale Mill - Cooling Towers,1100
228430,7650,Bartter Enterprises,SODHYP13-B,SODIUM HYPOCHLORITE 13% - BULK,2009-01,839.0,L,N106,37417,TURKEY PLANT- Steggles /Bartter,839


In [11]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
grouped_sales_data = sales_data.groupby(['Material', 'Delivery Date'])['Quantity in L'].sum().reset_index()
grouped_sales_data

,Material,Delivery Date,Quantity in L
0,ALUSUL08-1000,2010-01,16000.000
1,ALUSUL08-1000,2010-02,11000.000
2,ALUSUL08-1000,2010-03,26000.000
3,ALUSUL08-1000,2010-04,45000.000
4,ALUSUL08-1000,2010-05,25977.099
...,...,...,...
4142,SODHYP5-B,2020-12,4073.000
4143,SODHYP5-B,2021-01,3775.000
4144,SODHYP5-B,2021-02,3343.000
4145,SODHYP5-B,2021-03,3987.000


In [12]:
grouped_sales_data[grouped_sales_data['Material'] == 'ALUSUL08-1000']

,Material,Delivery Date,Quantity in L
0,ALUSUL08-1000,2010-01,16000.000
1,ALUSUL08-1000,2010-02,11000.000
2,ALUSUL08-1000,2010-03,26000.000
3,ALUSUL08-1000,2010-04,45000.000
4,ALUSUL08-1000,2010-05,25977.099
...,...,...,...
131,ALUSUL08-1000,2020-12,6053.435
132,ALUSUL08-1000,2021-01,2977.099
133,ALUSUL08-1000,2021-02,15816.794
134,ALUSUL08-1000,2021-03,9954.198
